In [ ]:
import tensorflow as tf
# from pycocotools.coco import COCO
import json
import pandas as pd
import PIL
import numpy

In [ ]:
val_annotation_path = '../../halte-data/coco/annotations/instances_val2017.json'
val_images_path = '../../halte-data/coco/val2017/'

In [ ]:
# coco=COCO(val_annotation_path)

In [ ]:
with open(val_annotation_path, 'r') as f:
    val_annotations = json.load(f)

In [ ]:
images_df = pd.DataFrame.from_dict(val_annotations['images'])
categories_df = pd.DataFrame.from_dict(val_annotations['categories'])
annotations_df = pd.DataFrame.from_dict(val_annotations['annotations'])

In [ ]:
person_annotations_df = annotations_df[annotations_df['category_id']==1]
person_annotations_df = person_annotations_df[person_annotations_df['iscrowd']==0]
person_annotations_df.reset_index(inplace=True, drop=True)
person_annotations_df.head()

In [ ]:
person_annotations_df['image_id'].value_counts()

In [ ]:
SEGMENTATION_PADDING = person_annotations_df['image_id'].value_counts().max()

In [ ]:
# images_df['image_path'] = images_df['id'].apply(lambda x : val_images_path + str(x).zfill(12) + '.jpg')
images_df['image_path'] = images_df['file_name'].apply(lambda x : val_images_path + x)
images_df.head()

In [ ]:
PIL.Image.open(images_df['image_path'][numpy.random.randint(0,len(images_df))])

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (255, 255))
#     img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img#, image_path

In [ ]:
person_annotations_df.head()

In [ ]:
def return_image_and_labels(img_index):
    row = images_df.loc[img_index]
    bboxes = person_annotations_df['bbox'][person_annotations_df['image_id']==row['id']].to_numpy()
#     labels = numpy.zeros(SEGMENTATION_PADDING, dtype=numpy.float32)
    labels = numpy.zeros((SEGMENTATION_PADDING, 4), dtype=numpy.float32)
    if len(bboxes):
        bboxes = numpy.stack(bboxes)
        labels[0:len(bboxes)] = bboxes
    img = load_image(row['image_path'])
    return img, labels

In [ ]:
i = numpy.random.randint(0,len(images_df))
print(images_df.loc[i])
bbox_count = len(person_annotations_df[person_annotations_df['image_id']==images_df.loc[i]['id']])
print('should find', bbox_count, 'boxes')
return_image_and_labels(i)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(images_df.index)

In [ ]:
dataset = dataset.map(lambda item1: tf.numpy_function(return_image_and_labels, [item1], [tf.float32, tf.float32]),
                      num_parallel_calls=tf.data.experimental.AUTOTUNE)


In [ ]:
global_batch_size=64
buffer_size=64
# Shuffle and batch
dataset = dataset.shuffle(buffer_size).batch(global_batch_size)
dataset = dataset.prefetch(buffer_size=16)#tf.data.experimental.AUTOTUNE)

In [ ]:
for i, batch in dataset:
    print(batch)
    break